In [ ]:
import os
from typing import TypedDict, List
from pydantic import BaseModel, Field, AliasChoices
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END
from constants import BASE_URL, API_KEY, MODEL_NAME
import pydantic

class PostBrief(BaseModel):
    # Если модель вернет {"brief": {"refined_topic": "..."}}, Alias поможет найти данные
    refined_topic: str = Field(
        validation_alias=AliasChoices('refined_topic', 'title', 'topic'),
        description="Уточненная тема с учетом контекста"
    )
    key_points: List[str] = Field(
        validation_alias=AliasChoices('key_points', 'objectives', 'main_ideas', 'guidelines'),
        description="Главные тезисы"
    )
    suggested_angle: str = Field(
        validation_alias=AliasChoices('suggested_angle', 'angle', 'tone', 'style'),
        description="Рекомендуемый фокус/ракурс поста"
    )
    prompt_for_writer: str = Field(
        validation_alias=AliasChoices('prompt_for_writer', 'unique_prompt', 'instruction'),
        description="Готовый промпт для следующего агента"
    )

    # Магия Pydantic: если модель вернула {"brief": {...}}, мы достаем данные оттуда
    @pydantic.model_validator(mode='before')
    @classmethod
    def unwrap_envelope(cls, data: any) -> any:
        if isinstance(data, dict) and len(data) == 1 and ('brief' in data or 'output' in data):
            return list(data.values())[0]
        return data

class PostState(TypedDict):
    user_topic: str         # Изначальная тема от пользователя
    website_context: str
    channel_archive: str
    objectives: str
    style_guide: str
    post_brief: PostBrief    # Результат работы планировщика
    final_variations: List[str] # 3 варианта поста

llm = ChatOpenAI(base_url=BASE_URL, api_key=API_KEY, model=MODEL_NAME, temperature=0.7)

objectives_path = '../DATA/post_requirements.md'

def context_loader(state: PostState):
    """Загружает базу знаний."""
    def read_file(path, default):
        return open(path, "r", encoding="utf-8").read() if os.path.exists(path) else default

    return {
        "website_context": read_file('../DATA/website_content.md', ""),
        "channel_archive": read_file('../DATA/posts.md', ""),
        "objectives": read_file(objectives_path, "")
    }

def strategist_brief_agent(state: PostState):
    structured_llm = llm.with_structured_output(PostBrief, method="json_mode")
    
    system_msg = SystemMessage(content=(
        "Ты — контент-стратег. Твоя задача: взять сырую тему пользователя и превратить её в крутой бриф.\n"
        "1. Проверь цели в objectives.\n"
        "2. Посмотри архив, чтобы не повторять то, что уже было.\n"
        "3. Сформулируй уникальный промпт для копирайтера.\n"
        "ВАЖНО: Выдавай JSON сразу с ключами 'refined_topic', 'key_points', 'suggested_angle', 'prompt_for_writer'.\n"
        "НЕ оборачивай ответ в дополнительные ключи типа 'brief' или 'post'."
    ))
    
    user_msg = HumanMessage(content=(
        f"ТЕМА: {state['user_topic']}\n"
        f"ЦЕЛИ: {state['objectives']}\n"
        f"АРХИВ: {state['channel_archive'][:2000]}\n"
        "Верни структурированный план поста."
    ))
    
    brief = structured_llm.invoke([system_msg, user_msg])
    return {"post_brief": brief}

def style_analyzer(state: PostState):
    """Выделяет ДНК стиля автора."""
    prompt = f"Выдели 5-7 правил стиля этого автора на основе постов:\n\n{state['channel_archive']}"
    res = llm.invoke(prompt)
    return {"style_guide": res.content}

def multi_writer_agent(state: PostState):
    """Пишет 3 разных варианта поста."""
    brief = state["post_brief"]
    variations = []
    
    # Мы сделаем один вызов, попросив LLM выдать 3 четко разделенных варианта
    # Это быстрее и дешевле, чем 3 отдельных вызова
    prompt = f"""
    Напиши 3 ВАРИАНТА поста для Telegram на тему: '{brief.refined_topic}'.
    
    ЗАДАНИЕ ИЗ БРИФА: {brief.prompt_for_writer}
    ТЕЗИСЫ: {', '.join(brief.key_points)}
    СТИЛЬ АВТОРА: {state['style_guide']}
    КОНТЕКСТ САЙТА: {state['website_context']}

    Требования к вариантам:
    Вариант 1: ЭКСПЕРТНЫЙ (глубокий анализ, польза).
    Вариант 2: ЛИЧНЫЙ/СТОРИТЕЛЛИНГ (через опыт, более мягкий стиль).
    Вариант 3: КРАТКИЙ/ТЕЗИСНЫЙ (инсайт, быстрое чтение).

    Разделяй варианты строкой '=== VARIATION_SEPARATOR ==='.
    """
    
    response = llm.invoke(prompt)
    content = response.content
    variations = [v.strip() for v in content.split('=== VARIATION_SEPARATOR ===')]
    
    return {"final_variations": variations}

builder = StateGraph(PostState)

builder.add_node("loader", context_loader)
builder.add_node("strategist", strategist_brief_agent)
builder.add_node("stylist", style_analyzer)
builder.add_node("writer", multi_writer_agent)

builder.set_entry_point("loader")
builder.add_edge("loader", "strategist")
builder.add_edge("strategist", "stylist")
builder.add_edge("stylist", "writer")
builder.add_edge("writer", END)

app = builder.compile()

topic_input = input("Введите тему поста: ") # Например: "Как я учу китайский с помощью AI"
    
print("\n🏗 Работаем над постом...")
result = app.invoke({"user_topic": topic_input})

print("\n" + "⭐" * 30)
print(f"СТРАТЕГИЧЕСКИЙ БРИФ:")
print(f"Уточненная тема: {result['post_brief'].refined_topic}")
print(f"Фокус: {result['post_brief'].suggested_angle}")
print("⭐" * 30 + "\n")

for i, post in enumerate(result['final_variations'], 1):
    print(f"--- ВАРИАНТ №{i} ---")
    print(post)
    print("\n" + "-"*40 + "\n")
    


🏗 Работаем над постом...

⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐
СТРАТЕГИЧЕСКИЙ БРИФ:
Уточненная тема: Как китайцы относятся к иностранцам: от стереотипов до реальности
Фокус: Развенчание мифов и стереотипов о восприятии иностранцев в Китае через личные истории и наблюдения, с добавлением легкого юмора и интересных фактов.
⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐

--- ВАРИАНТ №1 ---


----------------------------------------

--- ВАРИАНТ №2 ---
**Вариант 1: ЭКСПЕРТНЫЙ (глубокий анализ, польза)**

**Как китайцы относятся к иностранцам: от стереотипов до реальности**

Китай — страна контрастов, и отношение к иностранцам здесь не менее многогранно. С одной стороны, есть устойчивые стереотипы: "все иностранцы богаты", "они обязательно любят острое" или "не умеют пользоваться палочками". С другой — реальность часто опровергает эти клише.

**Культурные стереотипы и их влияние**
Многие китайцы до сих пор верят, что все иностранцы приезжают в Китай "зарабатывать большие деньги" или "учить английский". Это отча